# Imports

In [1]:
import numpy as np
import pandas as pd
import numexpr as ne

In [2]:
from simplenn.activations import LRelu
from simplenn.loss import Square
from simplenn.network import Network
from simplenn.network import Layer
from simplenn.optim import BackProp
from simplenn.optim import Genetic

In [3]:
%matplotlib

Using matplotlib backend: TkAgg


# Data

In [4]:
xs = np.array(
    [
        [0,0,0,0,0,0,0,0,0,1],
        [0,0,0,0,0,0,0,0,1,0],
        [0,0,0,0,0,0,0,1,0,0],
        [0,0,0,0,0,0,1,0,0,0],
        [0,0,0,0,0,1,0,0,0,0],
        [0,0,0,0,1,0,0,0,0,0],
        [0,0,0,1,0,0,0,0,0,0],
        [0,0,1,0,0,0,0,0,0,0],
        [0,1,0,0,0,0,0,0,0,0],
        [1,0,0,0,0,0,0,0,0,0]
    ],
    dtype='float64'
).T

ys = np.array(
    [
        [0,0,0,0],
        [0,0,0,1],
        [0,0,1,0],
        [0,0,1,1],
        [0,1,0,0],
        [0,1,0,1],
        [0,1,1,0],
        [0,1,1,1],
        [1,0,0,0],
        [1,0,0,1]
    ],
    dtype='float64'
).T

# Experiments

### Genetic Training

In [5]:
popSize = 50
nElitism = 0
generations = 500
tournamentSize = 5
mutationProba = 0.5
mutationScale = 0.01
mutationRelative = False
verboseFreq = 50
recordFreq = 50

In [8]:
net = Network(
    Square(),
    [
        Layer(xs.shape[0], 7, LRelu(), "kaiming"),
        Layer(7, ys.shape[0], LRelu(), "kaiming", True)
    ]
)

In [9]:
genetic = Genetic()
net = genetic.run(
    net, 
    xs, 
    ys, 
    popSize, 
    tournamentSize,
    nElitism, 
    mutationProba, 
    mutationScale, 
    mutationRelative, 
    generations, 
    verboseFreq, 
    recordFreq)

Generation 0: 5.62256482565631
Generation 50: 0.9387081647728822
Generation 100: 0.3504284777077845
Generation 150: 0.14155920458767648
Generation 200: 0.0176623529894496
Generation 250: 0.005299284168837626
Generation 300: 0.004981996290954782
Generation 350: 0.004356877559225041
Generation 400: 0.006473099771500696
Generation 450: 0.004855844414092482


In [10]:
net.forward(xs[:,:9]).round(2)

array([[ 0.02, -0.  , -0.  , -0.  , -0.  , -0.  , -0.  , -0.  ,  1.03],
       [-0.  , -0.  , -0.  , -0.  ,  0.99,  0.99,  1.03,  0.98, -0.  ],
       [-0.  , -0.  ,  0.99,  0.99, -0.  , -0.  ,  1.03,  1.04, -0.  ],
       [-0.  ,  1.01, -0.  ,  1.04, -0.  ,  1.06, -0.  ,  1.01, -0.  ]])

### Gradient Back-Propagation Training

In [11]:
lRate = 0.001
batchSize = xs.shape[1]
epochs = 100*1000
verboseFreq = 10*1000
recordFreq = 10*1000

In [12]:
net = Network(
    Square(),
    [
        Layer(xs.shape[0], 7, LRelu(0.05), "kaiming"),
        Layer(7, ys.shape[0], LRelu(0.05), "kaiming", True)
    ]
)

In [13]:
bp = BackProp()
bp.run(net, xs, ys, batchSize, lRate, epochs, verboseFreq, recordFreq)

Epoch 0: 8.3887049740548
Epoch 10000: 1.2480554566946411
Epoch 20000: 0.4190091611652043
Epoch 30000: 0.09569065505533962
Epoch 40000: 0.017792026459412795
Epoch 50000: 0.0038491470477052137
Epoch 60000: 0.001200422366522551
Epoch 70000: 0.0005427603076387476
Epoch 80000: 0.00032457090177839907
Epoch 90000: 0.00023379080888724798


In [14]:
net.forward(xs[:,:9]).round(2)

array([[-0.  ,  0.  ,  0.  , -0.  , -0.  ,  0.01, -0.  , -0.01,  1.  ],
       [-0.  ,  0.  ,  0.01, -0.  ,  1.  ,  1.  ,  1.  ,  1.  , -0.  ],
       [-0.  ,  0.  ,  1.  ,  1.  , -0.  ,  0.  ,  1.01,  0.99, -0.  ],
       [-0.  ,  1.  ,  0.  ,  1.  , -0.  ,  1.  ,  0.  ,  1.  , -0.  ]])

### Weights exploration

In [76]:
net.layers[0].W

array([[-0.50353252, -0.7438632 , -0.98149671, -0.52687593,  0.48586229,
        -0.19199454,  0.51301159, -0.23947582,  0.60441195,  0.71573741],
       [ 0.53088656,  0.67428931, -1.60380151, -1.0630602 , -0.13421359,
         0.08313299,  0.8626483 ,  1.00173144,  0.55501633,  0.71116043],
       [-2.09930766,  0.25535884,  0.31498162,  0.87877578, -1.11811146,
         0.42334603,  0.34976201,  0.8150063 ,  0.07705778,  0.40759563],
       [ 0.39736466,  0.33332055, -0.82968975, -0.14341508, -0.37056746,
         0.03519385, -0.28854572, -0.72349644, -1.37422008, -0.23418814],
       [ 0.34458334, -0.6615937 ,  0.54314591,  0.02720344,  0.29978532,
        -0.21600941,  0.67240304,  0.35797181,  0.32475208, -1.07483822],
       [-1.23480792, -0.6093184 ,  0.4661368 ,  0.26956696,  1.00189553,
         0.85654215, -0.50588299, -0.87564111,  0.3776834 ,  0.33993472],
       [ 0.76250696,  0.41734871, -1.40686184, -1.3727252 ,  0.00461855,
        -0.50132029, -0.411048  ,  0.33404063

In [355]:
net.layers[0].B

array([[-2.11372323e-01],
       [-1.27713963e+00],
       [ 2.16188538e-01],
       [-3.22498897e+00],
       [-3.92488673e+00],
       [-3.54475436e-02],
       [-3.73075906e+00],
       [-5.25937587e-01],
       [ 3.86582433e-03],
       [ 2.54796674e+00]])

In [356]:
net.layers[1].W

array([[-4.07636208, -0.13527109, -0.09725526, -3.87823325, -0.50223101,
        -5.1263356 , -0.56963821,  0.18459425,  0.57068787,  3.20610628],
       [-5.640668  ,  0.18420762, -4.2295531 , -0.64919038, -1.89953147,
         0.74924687,  1.28250049, -0.56709113,  0.51081547,  2.78299899],
       [ 5.05570252,  1.16659895, -3.27955841, -0.14404652,  4.30854958,
         0.55440625, -1.11615986, -0.29924194, -0.61474407, -0.46652216],
       [-3.24576778,  0.36209839, -0.68468027, -0.99633798, -0.08204685,
         1.52744847,  1.96596776,  0.46663185, -0.24713109,  0.69115127],
       [ 1.25171853, -0.98715313,  1.71372113,  1.68322207, -1.78604604,
         0.05520921, -3.16725124, -0.44534853, -0.00765816, -1.04281098],
       [ 2.52557192,  1.06029334, -0.93715661, -0.89932734, -1.24260812,
         5.31123914,  1.72137857, -0.16985234,  0.04836965, -0.74974051],
       [ 5.20460262,  0.64543036,  0.02838262, -2.36629912,  4.29518085,
        -3.20109392, -0.01082983,  1.35958486

In [357]:
net.layers[1].B

array([[-0.11100739],
       [-0.06765569],
       [-0.12821934],
       [-0.31170664],
       [-0.03842876],
       [-0.00603929],
       [-0.16960522],
       [ 0.5176651 ],
       [-0.69623932],
       [ 0.44453535]])

### Numexpr vs. Numpy

In [10]:
%%timeit
A = np.random.random((10000, 5000))
for _ in range(10):
    A = ne.evaluate("cos(A)")

2.46 s ± 183 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
A = np.random.random((10000, 5000))
for _ in range(10):
    A = np.cos(A)

6.07 s ± 155 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
